<a href="https://colab.research.google.com/github/karan2261/DistilBERT-Text-Classification/blob/main/DistilBERT_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading the trained model and evaluating the peroformance of the model on test dataset

In [ ]:
!pip install transformers

In [ ]:
# Importing necessary libraries
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import Trainer as TFTrainer
from transformers import TrainingArguments as TFTrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# Loading the datasets
train_df = pd.read_csv("Train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("Test.csv")

In [ ]:
# Map training labels to numeric values
train_df["label"] = train_df["label"].map({"positive": 1, "negative": 0})

# Correctting test labels
test_df["label"] = test_df["label"].replace(4, 1)

In [ ]:
# DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Tokenizing and encoding the datasets
def encode_data(data, max_length=128):
    return tokenizer(
        list(data["text"].values),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

train_encodings = encode_data(train_df)
val_encodings = encode_data(val_df)
test_encodings = encode_data(test_df)

In [ ]:
# Converting the labels to tensor format
train_labels = tf.convert_to_tensor(train_df["label"].values)
val_labels = tf.convert_to_tensor(val_df["label"].values)
test_labels = tf.convert_to_tensor(test_df["label"].values)

In [ ]:
# Loading the model
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
# Compiling the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

In [ ]:
# Training the model
model.fit(
    x=dict(train_encodings),
    y=train_labels,
    validation_data=(dict(val_encodings), val_labels),
    epochs=5,
    batch_size=64
)

Epoch 1/5
40/40 [==============================] - 95s 2s/step - loss: 0.4937 - accuracy: 0.7552 - val_loss: 0.6371 - val_accuracy: 0.6279
Epoch 2/5
40/40 [==============================] - 72s 2s/step - loss: 0.2652 - accuracy: 0.8952 - val_loss: 0.5962 - val_accuracy: 0.6908
Epoch 3/5
40/40 [==============================] - 72s 2s/step - loss: 0.1495 - accuracy: 0.9524 - val_loss: 0.6774 - val_accuracy: 0.6801
Epoch 4/5
40/40 [==============================] - 72s 2s/step - loss: 0.0656 - accuracy: 0.9828 - val_loss: 0.8297 - val_accuracy: 0.6833
Epoch 5/5
40/40 [==============================] - 64s 2s/step - loss: 0.0316 - accuracy: 0.9928 - val_loss: 0.8115 - val_accuracy: 0.6914


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
    x=dict(test_encodings),
    y=test_labels,
    batch_size=64
)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

40/40 [==============================] - 7s 126ms/step - loss: 0.8312 - accuracy: 0.6944
Test Loss: 0.8312, Test Accuracy: 0.6944


## Using val.csv dataset to fine tune the model.

In [ ]:
# Fine-tuning the model on validation data using a lower learning rate
fine_tune_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(optimizer=fine_tune_optimizer, loss=loss_fn, metrics=["accuracy"])

In [ ]:
# Fine-tuning the model with validation dataset
fine_tune_history = model.fit(
    x=dict(val_encodings),
    y=val_labels,
    epochs=3,
    batch_size=64,
    verbose=2
)

Epoch 1/3
157/157 - 110s - loss: 0.6470 - accuracy: 0.7019 - 110s/epoch - 699ms/step
Epoch 2/3
157/157 - 91s - loss: 0.5526 - accuracy: 0.7360 - 91s/epoch - 581ms/step
Epoch 3/3
157/157 - 91s - loss: 0.5187 - accuracy: 0.7529 - 91s/epoch - 580ms/step


## Evaluate the fine-tuned model on test dataset

In [ ]:
# Evaluate the fine-tuned model on the test dataset
test_loss, test_accuracy = model.evaluate(
    x=dict(test_encodings),
    y=test_labels,
    batch_size=64
)

print(f"Fine-Tuned Test Loss: {test_loss:.4f}, Fine-Tuned Test Accuracy: {test_accuracy:.4f}")

40/40 [==============================] - 8s 127ms/step - loss: 0.5106 - accuracy: 0.7660
Fine-Tuned Test Loss: 0.5106, Fine-Tuned Test Accuracy: 0.7660


## Additional approach for Fine-tuning

In [ ]:
# Unfreeze the last n layers of the model
n = 4
for layer in model.layers[-n:]:
    layer.trainable = True

In [ ]:
# Recompile the model with a lower learning rate for fine-tuning
fine_tune_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(optimizer=fine_tune_optimizer, loss=loss_fn, metrics=["accuracy"])

In [ ]:
# Fine-tune the model
fine_tune_history = model.fit(
    x=dict(val_encodings),
    y=val_labels,
    epochs=3,
    batch_size=64,
    verbose=2
)

Epoch 1/3
157/157 - 91s - loss: 0.4474 - accuracy: 0.7934 - 91s/epoch - 580ms/step
Epoch 2/3
157/157 - 91s - loss: 0.4347 - accuracy: 0.8059 - 91s/epoch - 580ms/step
Epoch 3/3
157/157 - 91s - loss: 0.4227 - accuracy: 0.8098 - 91s/epoch - 580ms/step


In [ ]:
# Evaluate the fine-tuned model on test data
test_loss, test_accuracy = model.evaluate(
    x=dict(test_encodings),
    y=test_labels,
    batch_size=64
)

print(f"Fine-Tuned Test Loss: {test_loss:.4f}, Fine-Tuned Test Accuracy: {test_accuracy:.4f}")

40/40 [==============================] - 5s 127ms/step - loss: 0.4439 - accuracy: 0.8112
Fine-Tuned Test Loss: 0.4439, Fine-Tuned Test Accuracy: 0.8112


## Discussion

* Initially, the model achieved 69.4% accuracy on the test set. After fine-tuning with val.csv at a lower learning rate, the test accuracy improved to 76.6%. Further fine-tuning by unfreezing additional layers pushed it up to 81.1%.

* Fine-tuning significantly boosted performance.




**For Further Improvments :**

* Trying different learning rates, batch sizes and epochs could get better results.

* I can try experimenting with unfreezing more layers that may enhance the model's ability to adapt.

* Using additional data relevant to the target domain could improve the model’s generalization, especially if the validation or test set distribution differs from the training data.

**Conclusion:** Fine-tuning the model on val.csv improved accuracy, but further adjustments could enhance it even more.